In [36]:
from mem0 import AsyncMemoryClient
import os

In [37]:
os.environ["MEM0_API_KEY"] = "<api-key>"


In [38]:
client = AsyncMemoryClient()

import google.generativeai as genai
import json
from typing import List, Dict
import asyncio


In [39]:
# Configure Gemini API (you'll need to set your API key)
# Get your API key from https://makersuite.google.com/app/apikey
GEMINI_API_KEY = "<api-key>"  # Replace with your actual API key
GEMINI_MODEL='gemini-2.5-flash'
genai.configure(api_key=GEMINI_API_KEY)


In [82]:
# System prompt for generating memory snippets from page content
MEMORY_SNIPPET_PROMPT = """
You are an AI assistant that creates concise, valuable memory snippets from web page content.

Your task is to analyze the given page content and extract 3-5 key memory snippets that capture:
1. Main concepts, facts, or insights
2. User preferences or interests mentioned
3. Important methodologies or approaches
4. Key takeaways or conclusions
5. Any personal context or examples

Each memory snippet should be:
- 1-2 sentences maximum
- Specific rather than generic
- Useful for future reference
- Written in third person when applicable (e.g., "User read that..." or "User visited...")

Return the snippets as a JSON array of strings.

Page Content:
{content}

Return the respone in valid JSON format, no additional text not even a code block specifying json
"""


In [65]:
# System prompt for rephrasing content based on existing memory
CONTENT_REPHRASING_PROMPT = """
You are an AI assistant that rephrases web page content based on a user's existing knowledge and interests.

Given the user's existing memories and the current page content, your task is to:

1. **Identify overlaps**: What concepts in the current content does the user already know?
2. **Highlight gaps**: What new information should be emphasized for the user?
3. **Personalize**: Adjust the content based on the user's interests and background.

Existing User Memory:
{existing_memory}

Current Page Content:
{content}

Instructions:
- Rewrite the content to be more personalized and relevant
- Use phrases like "Building on what you already know about...", "Since you're interested in...", "This expands on your knowledge of..."
- Emphasize new concepts that complement the user's existing knowledge
- Skip or briefly mention concepts the user already understands well
- Maintain the original structure but adjust focus and emphasis

Return the rephrased content:
"""


In [66]:
# Sample blog posts for testing (different and overlapping genres)
SAMPLE_BLOG_POSTS = {
    "tech_productivity": """
    # The Art of Deep Work in the Digital Age
    
    In today's hyper-connected world, the ability to focus without distraction on cognitively demanding tasks has become increasingly rare. Cal Newport calls this skill "deep work" - the ability to focus without distraction on a cognitively demanding task.
    
    ## Why Deep Work Matters
    
    Deep work is valuable because it allows you to:
    - Learn complex skills faster
    - Produce better quality output
    - Create more meaningful work
    - Stand out in an increasingly competitive job market
    
    ## Strategies for Cultivating Deep Work
    
    1. **Time Blocking**: Schedule specific blocks of time for deep work, treating them as sacred appointments
    2. **Environment Design**: Create a dedicated workspace that signals to your brain it's time to focus
    3. **Digital Minimalism**: Remove distracting apps and notifications during work sessions
    4. **Rhythmic Approach**: Establish a consistent routine for deep work sessions
    
    ## The 4-Hour Rule
    
    Research suggests that most people can sustain about 4 hours of deep work per day. Novices might start with 1-2 hours and gradually build up their capacity.
    
    Remember: Deep work is a skill that requires practice. Like physical exercise, your ability to concentrate deeply will improve with consistent effort.
    """,
    
    "health_fitness": """
    # Building Sustainable Exercise Habits: A Science-Based Approach
    
    Creating lasting fitness habits isn't about willpower - it's about understanding the psychology of behavior change and applying proven strategies to make exercise stick.
    
    ## The Psychology of Habit Formation
    
    According to James Clear's research, habits consist of four components:
    1. **Cue**: The trigger that initiates the behavior
    2. **Craving**: The motivation behind every habit
    3. **Response**: The actual habit you perform
    4. **Reward**: The benefit you gain from doing the habit
    
    ## Start Small, Think Big
    
    The key to building exercise habits is starting ridiculously small. Instead of committing to hour-long gym sessions, begin with:
    - 5 pushups after waking up
    - A 10-minute walk after lunch
    - 2 minutes of stretching before bed
    
    ## The Power of Environment Design
    
    Your environment shapes your behavior more than motivation does. Set up your space to make exercise easier:
    - Keep workout clothes visible
    - Set up your home gym space the night before
    - Join a gym close to your home or work
    
    ## Tracking and Accountability
    
    What gets measured gets managed. Use simple tracking methods:
    - Calendar marking for consistency
    - Fitness apps for progress
    - Workout buddy for accountability
    
    Remember: Consistency beats intensity. It's better to exercise for 10 minutes daily than 2 hours once a week.
    """,
    
    "productivity_mindfulness": """
    # Mindful Productivity: Balancing Efficiency with Well-being
    
    The traditional productivity paradigm focuses solely on doing more, faster. But what if there's a better way - one that combines efficiency with mindfulness and well-being?
    
    ## Rethinking Productivity
    
    Mindful productivity isn't about cramming more tasks into your day. It's about:
    - Being intentional with your time and energy
    - Focusing on what truly matters
    - Creating sustainable work practices
    - Maintaining mental and emotional well-being
    
    ## Core Principles
    
    ### Single-Tasking Over Multi-tasking
    Research consistently shows that multitasking reduces efficiency and increases errors. Instead:
    - Focus on one task at a time
    - Use time-blocking techniques
    - Minimize task-switching
    
    ### Quality Over Quantity
    - Prioritize high-impact activities
    - Learn to say no to non-essential requests
    - Focus on outcomes, not just activity
    
    ### Mindful Breaks
    Taking breaks isn't laziness - it's essential for sustained performance:
    - 5-minute meditation sessions
    - Short walks in nature
    - Mindful breathing exercises
    
    ## Implementation Strategies
    
    1. **Morning Ritual**: Start your day with intention-setting and mindfulness
    2. **Energy Management**: Schedule demanding tasks during your peak energy hours
    3. **Regular Check-ins**: Pause periodically to assess your mental state and priorities
    4. **Evening Reflection**: End your day by reviewing accomplishments and planning tomorrow
    
    The goal isn't to be busy - it's to be effective while maintaining your well-being.
    """
}


In [67]:
# Initialize Gemini model
model = genai.GenerativeModel(GEMINI_MODEL)

# System prompt for extracting key topics for memory search
TOPIC_EXTRACTION_PROMPT = """
You are an AI assistant that extracts key topics and themes from content for semantic search.

Analyze the given content and extract 3-5 key topics/themes that would be useful for searching related memories. Focus on:
1. Main subject areas (e.g., "productivity", "exercise", "habits")
2. Specific methodologies or concepts (e.g., "deep work", "time blocking", "habit formation")
3. Related skills or areas (e.g., "focus techniques", "behavior change", "mindfulness")

Return the topics as a JSON array of strings, each topic should be 1-4 words.

Content:
{content}

Return only the JSON array, no additional text not even a code block specifying json.
"""

async def extract_content_topics(content: str) -> List[str]:
    """Extract key topics from content for targeted memory search"""
    try:
        prompt = TOPIC_EXTRACTION_PROMPT.format(content=content)
        response = model.generate_content(prompt)
        topics = json.loads(response.text)
        return topics
    except Exception as e:
        print(f"Error extracting topics: {e}")
        return []

async def search_relevant_memories(content: str, user_id: str, max_memories: int = 10) -> List[Dict]:
    """Search for memories relevant to the current page content"""
    try:
        # Extract key topics from the content
        topics = await extract_content_topics(content)
        print(f"🔍 Extracted topics for memory search: {topics}")
        
        all_relevant_memories = []
        seen_memory_ids = set()
        
        # Search for memories related to each topic
        for topic in topics:
            try:
                memories = await client.search(topic, user_id=user_id)
                # Add unique memories (avoid duplicates)
                for mem in memories:
                    if mem['id'] not in seen_memory_ids and mem['score'] > 0.3:  # Minimum relevance threshold
                        all_relevant_memories.append(mem)
                        seen_memory_ids.add(mem['id'])
            except Exception as e:
                print(f"Error searching for topic '{topic}': {e}")
                continue
        
        # Sort by relevance score and limit results
        all_relevant_memories.sort(key=lambda x: x['score'], reverse=True)
        relevant_memories = all_relevant_memories[:max_memories]
        
        print(f"📚 Found {len(relevant_memories)} relevant memories (from {len(all_relevant_memories)} total matches)")
        return relevant_memories
        
    except Exception as e:
        print(f"Error searching relevant memories: {e}")
        return []

async def generate_memory_snippets(content: str) -> List[str]:
    """Generate memory snippets from page content using Gemini"""
    try:
        prompt = MEMORY_SNIPPET_PROMPT.format(content=content)
        response = model.generate_content(prompt)
        
        # Parse JSON response
        snippets = json.loads(response.text)
        return snippets
    except Exception as e:
        print(f"Error generating memory snippets: {e}")
        return []

async def rephrase_content_with_memory(content: str, existing_memories: List[Dict]) -> str:
    """Rephrase content based on existing user memories"""
    try:
        # Format existing memories into a readable string
        memory_text = "\n".join([f"- {mem['memory']} (relevance: {mem['score']:.2f})" for mem in existing_memories])
        
        prompt = CONTENT_REPHRASING_PROMPT.format(
            existing_memory=memory_text,
            content=content
        )
        
        response = model.generate_content(prompt)
        return response.text
    except Exception as e:
        print(f"Error rephrasing content: {e}")
        return content  # Return original content if rephrasing fails


In [74]:
async def process_page_content(content: str, user_id: str, add_to_memory: bool = True):
    """
    Main function to process page content:
    1. Generate memory snippets
    2. Add snippets to memory (if requested)
    3. Search for relevant existing memories (instead of getting all)
    4. Rephrase content based on relevant memories
    """
    print("\n🧠 Searching for relevant memories...")
    relevant_memories = await search_relevant_memories(content, user_id)

    print("🔍 Generating memory snippets...")
    snippets = await generate_memory_snippets(content)
    
    print(f"📝 Generated {len(snippets)} memory snippets:")
    for i, snippet in enumerate(snippets, 1):
        print(f"  {i}. {snippet}")
    
    if add_to_memory and snippets:
        print("\n💾 Adding snippets to memory...")
        for snippet in snippets:
            messages = [{"role": "user", "content": snippet}]
            result = await client.add(messages, user_id=user_id)
            print(f"  ✅ Added: {snippet}")
    
    
    if relevant_memories:
        print(f"📚 Found {len(relevant_memories)} relevant memories:")
        for mem in relevant_memories[:3]:  # Show top 3
            print(f"  📌 {mem['memory'][:80]}... (score: {mem['score']:.3f})")
        
        print("\n🔄 Rephrasing content based on relevant memories...")
        rephrased_content = await rephrase_content_with_memory(content, relevant_memories)
        
        print("\n" + "="*50)
        print("📄 REPHRASED CONTENT:")
        print("="*50)
        print(rephrased_content)
        print("="*50)
        
        return {
            "snippets": snippets,
            "relevant_memories": relevant_memories,
            "rephrased_content": rephrased_content
        }
    else:
        print("ℹ️  No relevant memories found. Content will not be rephrased.")
        return {
            "snippets": snippets,
            "relevant_memories": [],
            "rephrased_content": content
        }


## Testing the Memory-Enhanced Reading System

Now let's test our system with the sample blog posts. We'll:

1. **First article**: Process tech/productivity content and build initial memories
2. **Second article**: Process health/fitness content (different genre) 
3. **Third article**: Process productivity/mindfulness content (overlapping with first article)

This will demonstrate how the system:
- Extracts relevant memory snippets
- Builds a knowledge base over time
- Rephrases content based on what the user already knows

**Note**: Make sure to set your Gemini API key in the cell above before running the tests!


In [83]:
# Test 1: Process the first article (Deep Work - Tech/Productivity)
print("🚀 PROCESSING FIRST ARTICLE: Deep Work")
print("="*60)

result1 = await process_page_content(
    content=SAMPLE_BLOG_POSTS["tech_productivity"],
    user_id="anunay",
    add_to_memory=True
)


🚀 PROCESSING FIRST ARTICLE: Deep Work

🧠 Searching for relevant memories...
🔍 Extracted topics for memory search: ['Deep Work', 'Focus Techniques', 'Time Blocking', 'Digital Minimalism', 'Productivity']
📚 Found 10 relevant memories (from 15 total matches)
🔍 Generating memory snippets...
📝 Generated 3 memory snippets:
  1. User learned that 'deep work' is the ability to focus without distraction on cognitively demanding tasks, a valuable skill for accelerated learning and higher quality output.
  2. The article outlines key strategies for cultivating deep work, including time blocking, designing a dedicated workspace, practicing digital minimalism, and establishing a rhythmic routine.
  3. User noted that most people can sustain about 4 hours of deep work per day, emphasizing that deep work is a skill that improves with consistent practice, similar to physical exercise.

💾 Adding snippets to memory...


/var/folders/qc/t0gsrkxn5d1gdxwym6tpywb80000gn/T/ipykernel_69056/2701878146.py:23: DeprecationWarning: output_format='v1.0' is deprecated therefore setting it to 'v1.1' by default. Check out the docs for more information: https://docs.mem0.ai/platform/quickstart#4-1-create-memories
  result = await client.add(messages, user_id=user_id)


  ✅ Added: User learned that 'deep work' is the ability to focus without distraction on cognitively demanding tasks, a valuable skill for accelerated learning and higher quality output.
  ✅ Added: The article outlines key strategies for cultivating deep work, including time blocking, designing a dedicated workspace, practicing digital minimalism, and establishing a rhythmic routine.
  ✅ Added: User noted that most people can sustain about 4 hours of deep work per day, emphasizing that deep work is a skill that improves with consistent practice, similar to physical exercise.
📚 Found 10 relevant memories:
  📌 Has learned that 'deep work' is the ability to focus without distraction on cogn... (score: 0.670)
  📌 Believes deep work is essential for faster learning, higher quality output, and ... (score: 0.633)
  📌 Deep work skill improves with consistent practice, similar to exercise... (score: 0.609)

🔄 Rephrasing content based on relevant memories...

📄 REPHRASED CONTENT:
Here's a rephras

In [77]:
# Test 2: Process the second article (Exercise Habits - Health/Fitness)
print("\n🏃 PROCESSING SECOND ARTICLE: Exercise Habits")
print("="*60)

result2 = await process_page_content(
    content=SAMPLE_BLOG_POSTS["health_fitness"],
    user_id="anunay",
    add_to_memory=True
)



🏃 PROCESSING SECOND ARTICLE: Exercise Habits

🧠 Searching for relevant memories...
🔍 Extracted topics for memory search: ['Exercise Habits', 'Habit Formation', 'Behavior Change', 'Environment Design', 'Tracking Accountability']
📚 Found 9 relevant memories (from 9 total matches)
🔍 Generating memory snippets...
📝 Generated 5 memory snippets:
  1. I learned that building sustainable exercise habits focuses on understanding behavior psychology and applying consistent, small steps, rather than relying solely on willpower.
  2. I understand that habits follow a 4-part loop: Cue, Craving, Response, and Reward, which I can use to design my exercise routine.
  3. To build new exercise habits, I should start ridiculously small, like 5 pushups or a 10-minute walk, to ensure consistency.
  4. I should design my environment to make exercise easier, such as keeping workout clothes visible or setting up my home gym in advance.
  5. I need to remember that consistency is more important than intensity

/var/folders/qc/t0gsrkxn5d1gdxwym6tpywb80000gn/T/ipykernel_69056/2701878146.py:23: DeprecationWarning: output_format='v1.0' is deprecated therefore setting it to 'v1.1' by default. Check out the docs for more information: https://docs.mem0.ai/platform/quickstart#4-1-create-memories
  result = await client.add(messages, user_id=user_id)


  ✅ Added: I learned that building sustainable exercise habits focuses on understanding behavior psychology and applying consistent, small steps, rather than relying solely on willpower.
  ✅ Added: I understand that habits follow a 4-part loop: Cue, Craving, Response, and Reward, which I can use to design my exercise routine.
  ✅ Added: To build new exercise habits, I should start ridiculously small, like 5 pushups or a 10-minute walk, to ensure consistency.
  ✅ Added: I should design my environment to make exercise easier, such as keeping workout clothes visible or setting up my home gym in advance.
  ✅ Added: I need to remember that consistency is more important than intensity; exercising for 10 minutes daily is better than 2 hours once a week.
📚 Found 9 relevant memories:
  📌 Should start building new exercise habits by committing to small actions like 5 ... (score: 0.653)
  📌 Believes consistency in exercise habits is more important than intensity... (score: 0.630)
  📌 Learned that

In [78]:
# Test 3: Process the third article (Mindful Productivity - overlapping with first)
print("\n🧘 PROCESSING THIRD ARTICLE: Mindful Productivity")
print("="*60)

result3 = await process_page_content(
    content=SAMPLE_BLOG_POSTS["productivity_mindfulness"],
    user_id="anunay",
    add_to_memory=True
)



🧘 PROCESSING THIRD ARTICLE: Mindful Productivity

🧠 Searching for relevant memories...
🔍 Extracted topics for memory search: ['Mindful Productivity', 'Well-being', 'Time Management', 'Focus Techniques', 'Mindfulness']
📚 Found 10 relevant memories (from 12 total matches)
🔍 Generating memory snippets...
📝 Generated 4 memory snippets:
  1. I learned that mindful productivity balances efficiency with well-being by focusing on intentionality, sustainable practices, and mental health, rather than just doing more.
  2. I should prioritize single-tasking over multitasking, using techniques like time-blocking to minimize task-switching and improve efficiency.
  3. I need to incorporate mindful breaks, such as 5-minute meditations or short walks, as they are essential for sustained performance and well-being.
  4. I can implement mindful productivity by starting with a morning ritual, scheduling demanding tasks during peak energy hours, and reflecting in the evening.

💾 Adding snippets to memor

/var/folders/qc/t0gsrkxn5d1gdxwym6tpywb80000gn/T/ipykernel_69056/2701878146.py:23: DeprecationWarning: output_format='v1.0' is deprecated therefore setting it to 'v1.1' by default. Check out the docs for more information: https://docs.mem0.ai/platform/quickstart#4-1-create-memories
  result = await client.add(messages, user_id=user_id)


  ✅ Added: I learned that mindful productivity balances efficiency with well-being by focusing on intentionality, sustainable practices, and mental health, rather than just doing more.
  ✅ Added: I should prioritize single-tasking over multitasking, using techniques like time-blocking to minimize task-switching and improve efficiency.
  ✅ Added: I need to incorporate mindful breaks, such as 5-minute meditations or short walks, as they are essential for sustained performance and well-being.
  ✅ Added: I can implement mindful productivity by starting with a morning ritual, scheduling demanding tasks during peak energy hours, and reflecting in the evening.
📚 Found 10 relevant memories:
  📌 Key strategies for cultivating deep work include time blocking, designing a dedi... (score: 0.518)
  📌 Has learned that 'deep work' is the ability to focus without distraction on cogn... (score: 0.473)
  📌 Believes deep work is essential for faster learning, higher quality output, and ... (score: 0.455)

In [79]:
# Let's also test the search functionality
print("\n🔍 TESTING MEMORY SEARCH FUNCTIONALITY")
print("="*60)

# Search for productivity-related memories
productivity_memories = await client.search("productivity and focus techniques", user_id="anunay")
print(f"Found {len(productivity_memories)} memories related to productivity:")
for mem in productivity_memories[:3]:  # Show top 3
    print(f"  📌 {mem['memory']} (score: {mem['score']:.3f})")

print("\n")

# Search for habit-related memories
habit_memories = await client.search("building habits and consistency", user_id="anunay")
print(f"Found {len(habit_memories)} memories related to habits:")
for mem in habit_memories[:3]:  # Show top 3
    print(f"  📌 {mem['memory']} (score: {mem['score']:.3f})")



🔍 TESTING MEMORY SEARCH FUNCTIONALITY
Found 10 memories related to productivity:
  📌 Has learned that mindful productivity balances efficiency with well-being by focusing on intentionality, sustainable practices, and mental health rather than just doing more (score: 0.583)
  📌 Has learned that 'deep work' is the ability to focus without distraction on cognitively demanding tasks (score: 0.536)
  📌 Key strategies for cultivating deep work include time blocking, designing a dedicated workspace, practicing digital minimalism, and establishing a consistent rhythmic routine (score: 0.530)


Found 10 memories related to habits:
  📌 Believes consistency in exercise habits is more important than intensity (score: 0.618)
  📌 Learned that building sustainable exercise habits relies on understanding behavior psychology, specifically James Clear's habit loop: Cue, Craving, Response, and Reward (score: 0.573)
  📌 Can improve exercise consistency by designing environment to make workouts easier, su

In [80]:
# Demonstration: Compare old vs new memory search approach
print("🔬 COMPARING MEMORY SEARCH APPROACHES")
print("="*60)

# Test content - a new article about time management
test_content = """
# Effective Time Management for Remote Workers

Remote work presents unique challenges for time management. Without the structure of an office environment, many remote workers struggle with productivity and work-life balance.

## Key Strategies

1. **Pomodoro Technique**: Work in 25-minute focused bursts with 5-minute breaks
2. **Time Blocking**: Allocate specific time slots for different types of work
3. **Environment Optimization**: Create a dedicated workspace that minimizes distractions
4. **Regular Breaks**: Step away from your desk to maintain mental clarity

The key is finding what works for your personal rhythm and sticking to it consistently.
"""

print("📄 Test content: Time Management for Remote Workers")
print("\n1️⃣ OLD APPROACH: Getting ALL memories")
all_memories = await client.get_all(user_id="anunay")
print(f"   📊 Retrieved {len(all_memories)} total memories")

print("\n2️⃣ NEW APPROACH: Targeted memory search")
relevant_memories = await search_relevant_memories(test_content, user_id="anunay", max_memories=5)
print(f"   📊 Found {len(relevant_memories)} relevant memories")

print("\n💡 EFFICIENCY COMPARISON:")
print(f"   🔹 Old approach: {len(all_memories)} memories processed")
print(f"   🔹 New approach: {len(relevant_memories)} memories processed")
print(f"   🔹 Efficiency gain: {((len(all_memories) - len(relevant_memories)) / len(all_memories) * 100):.1f}% reduction in irrelevant data")


🔬 COMPARING MEMORY SEARCH APPROACHES
📄 Test content: Time Management for Remote Workers

1️⃣ OLD APPROACH: Getting ALL memories
   📊 Retrieved 19 total memories

2️⃣ NEW APPROACH: Targeted memory search
🔍 Extracted topics for memory search: ['Time Management', 'Remote Work Productivity', 'Pomodoro Technique', 'Time Blocking', 'Work-Life Balance']
📚 Found 5 relevant memories (from 18 total matches)
   📊 Found 5 relevant memories

💡 EFFICIENCY COMPARISON:
   🔹 Old approach: 19 memories processed
   🔹 New approach: 5 memories processed
   🔹 Efficiency gain: 73.7% reduction in irrelevant data


## Analysis and Chrome Extension Integration

Based on the testing above, here's how this would work in your Chrome extension:

### Chrome Extension Workflow:

1. **Content Extraction**: Use content scripts to extract the main article content from the current page
2. **Memory Processing**: Send the content to your backend (or use Google Chrome's built-in AI APIs)
3. **Memory Management**: 
   - Generate snippets using Gemini
   - Store/retrieve memories using Mem0
   - Rephrase content based on existing memories
4. **UI Display**: Show the rephrased content in a side panel or overlay

### Key Benefits Observed:

- **Personalization**: Content gets rephrased based on what user already knows
- **Knowledge Building**: Memories accumulate over time across different topics
- **Smart Recommendations**: Related memories help surface connections between topics
- **Efficiency**: Users can focus on new information rather than re-reading familiar concepts


In [81]:
# Optional: Clean up test memories (uncomment if you want to reset)
# print("🧹 Cleaning up test memories...")
# await client.delete_all(user_id="anunay")
# print("✅ All memories deleted")

# Show final memory count
final_memories = await client.get_all(user_id="anunay")
print(f"📊 Final memory count: {len(final_memories)} memories stored")
print("\n🎉 Testing complete! The system successfully:")
print("  ✅ Generated memory snippets from content")
print("  ✅ Stored memories in Mem0")
print("  ✅ Retrieved relevant memories")
print("  ✅ Rephrased content based on existing knowledge")
print("  ✅ Demonstrated cross-topic memory connections")


📊 Final memory count: 19 memories stored

🎉 Testing complete! The system successfully:
  ✅ Generated memory snippets from content
  ✅ Stored memories in Mem0
  ✅ Retrieved relevant memories
  ✅ Rephrased content based on existing knowledge
  ✅ Demonstrated cross-topic memory connections
